# XOR(exclusive OR) 문제해결

## non-linear
𝜙(⋅):𝑅𝐷→𝑅𝑀

벡터를 내적(inner product) 한 값  𝜙(𝑥𝑖)𝑇𝜙(𝑥𝑗) 를 하나의 함수로 나타낼 수 있다.

𝑘(𝑥𝑖,𝑥𝑗)=𝜙(𝑥𝑖)𝑇𝜙(𝑥𝑗)
 
이러한 함수를 커널(kernel)이라고 한다.

## 많이 사용되는 커널¶

선형 서포트 벡터 머신
𝑘(𝑥1,𝑥2)=𝑥𝑇1𝑥2
 
다항 커널 (Polynomial Kernel)
𝑘(𝑥1,𝑥2)=(𝛾(𝑥𝑇1𝑥2)+𝜃)𝑑
 
RBF(Radial Basis Function) 또는 가우시안 커널(Gaussian Kernel)
𝑘(𝑥1,𝑥2)=exp(−𝛾||𝑥1−𝑥2||2)
 
시그모이드 커널 (Sigmoid Kernel)
𝑘(𝑥1,𝑥2)=tanh(𝛾(𝑥𝑇1𝑥2)+𝜃)

## RBF 커널¶
RBF 커널은 가우시안 커널이라고도 한다. 문제를 간단하게 하기 위해 다음과 같이 가정한다.

𝛾=12
 
‖𝑥1‖=‖𝑥2‖=1

# 모형 최적화

Scikit-Learn에서는 다음과 같은 모형 최적화 도구를 지원한다.

- validation_curve
    단일 하이퍼 파라미터 최적화
- GridSearchCV
    그리드를 사용한 복수 하이퍼 파라미터 최적화
- ParameterGrid
    복수 파라미터 최적화용 그리드

## 병렬 처리

GridSearchCV 명령에는 n_jobs 라는 인수가 있다. 디폴트 값은 1인데 이 값을 증가시키면 내부적으로 멀티 프로세스를 사용하여 그리드서치를 수행한다. 만약 CPU 코어의 수가 충분하다면 n_jobs를 늘릴 수록 속도가 증가한다.

# 비대칭 데이터 문제

데이터 클래스 비율이 너무 차이가 나면(highly-imbalanced data) 단순히 우세한 클래스를 택하는 모형의 정확도가 높아지므로 모형의 성능판별이 어려워진다. 즉, 정확도(accuracy)가 높아도 데이터 갯수가 적은 클래스의 재현율(recall-rate)이 급격히 작아지는 현상이 발생할 수 있다.

이렇게 각 클래스에 속한 데이터의 갯수의 차이에 의해 발생하는 문제들을 비대칭 데이터 문제(imbalanced data problem)이라고 한다.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.svm import SVC


def classification_result(n0, n1, title=""):
    rv1 = sp.stats.multivariate_normal([-1, 0], [[1, 0], [0, 1]])
    rv2 = sp.stats.multivariate_normal([+1, 0], [[1, 0], [0, 1]])
    X0 = rv1.rvs(n0, random_state=0)
    X1 = rv2.rvs(n1, random_state=0)
    X = np.vstack([X0, X1])
    y = np.hstack([np.zeros(n0), np.ones(n1)])

    x1min = -4; x1max = 4
    x2min = -2; x2max = 2
    xx1 = np.linspace(x1min, x1max, 1000)
    xx2 = np.linspace(x2min, x2max, 1000)
    X1, X2 = np.meshgrid(xx1, xx2)

    plt.contour(X1, X2, rv1.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")
    plt.contour(X1, X2, rv2.pdf(np.dstack([X1, X2])), levels=[0.05], linestyles="dashed")

    model = SVC(kernel="linear", C=1e4, random_state=0).fit(X, y)
    Y = np.reshape(model.predict(np.array([X1.ravel(), X2.ravel()]).T), X1.shape)
    plt.scatter(X[y == 0, 0], X[y == 0, 1], marker='x', label="0 클래스")
    plt.scatter(X[y == 1, 0], X[y == 1, 1], marker='o', label="1 클래스")
    plt.contour(X1, X2, Y, colors='k', levels=[0.5])
    y_pred = model.predict(X)
    plt.xlim(-4, 4)
    plt.ylim(-3, 3)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title(title)
    
    return model, X, y, y_pred
    
plt.subplot(121)
model1, X1, y1, y_pred1 = classification_result(200, 200, "대칭 데이터 (5:5)")
plt.subplot(122)
model2, X2, y2, y_pred2 = classification_result(200, 20, "비대칭 데이터 (9:1)")
plt.tight_layout()
plt.show()

In [2]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y1, y_pred1))
print(classification_report(y2, y_pred2))

              precision    recall  f1-score   support

         0.0       0.86      0.83      0.84       200
         1.0       0.84      0.86      0.85       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97       200
         1.0       0.75      0.60      0.67        20

    accuracy                           0.95       220
   macro avg       0.86      0.79      0.82       220
weighted avg       0.94      0.95      0.94       220



## 해결 방법¶
비대칭 데이터는 다수 클래스 데이터에서 일부만 사용하는 언더 샘플링이나 소수 클래스 데이터를 증가시키는 오버 샘플링을 사용하여 데이터 비율을 맞추면 정밀도(precision)가 향상된다.

- 오버샘플링(Over-Sampling)
- 언더샘플링(Under-Sampling)
- 복합샘플링(Combining Over-and Under-Sampling)

### Tomek’s link method
- 토멕링크(Tomek’s link)란 서로 다른 클래스에 속하는 한 쌍의 데이터  (𝑥+,𝑥−) 로 서로에게 더 가까운 다른 데이터가 존재하지 않는 것이다.

# 특징 선택¶
- 분산선택
- 단일변수선택
- 다른 모형을 이용한 특성 중요도 계산

# 대규모 데이터 학습 Big Data